<a href="https://colab.research.google.com/github/robertmaxwilliams/talking-statues/blob/master/colab/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Make sure google drive is connected and data is accessible.

In [1]:
!nvidia-smi

Fri Mar  6 17:25:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
%tensorflow_version 1.x
!pip install -q gpt_2_simple 

import gpt_2_simple as gpt2
import os

class bcolors:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def generate_text(prefix):
    prefix = prefix[:-100]
    output = gpt2.generate(
        sess,
        prefix=prefix,
        include_prefix=False,
        return_as_list=True,
        length=100,
        batch_size=5,
        nsamples=5,
    )
    # print(f'\n\n{bcolors.OKGREEN}{prefix} {bcolors.ENDC}{output}\n')
    output = [x.lstrip(prefix) for x in output]
    return output


def finetune_file(path, run_name):
  gpt2.finetune(sess,
    dataset=path,
    model_name='124M',
    steps=100,
    # restore_from='fresh',
    run_name=run_name,
    print_every=10,
    sample_every=200,
    save_every=100,
  )


data_path = "/content/drive/My Drive/GRIMM/";
sess = gpt2.start_tf_sess()

for i, f in enumerate(os.listdir(data_path)):
  file_path = data_path + f
  run_name = "finetune_" + str(i)
  print(file_path)
  finetune_file(file_path, run_name)
  gpt2.copy_checkpoint_to_gdrive(run_name=run_name)



/content/drive/My Drive/GRIMM/THE_WOLF_AND_THE_SEVEN_LITTLE_KIDS.txt
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/finetune_0/model-10
INFO:tensorflow:Restoring parameters from checkpoint/finetune_0/model-10


100%|██████████| 1/1 [00:00<00:00, 127.15it/s]

Loading dataset...
dataset has 1409 tokens
Training...


[20 | 19.96] loss=0.08 avg=0.08
[30 | 32.77] loss=0.02 avg=0.05
[40 | 45.58] loss=0.03 avg=0.04
[50 | 58.42] loss=0.01 avg=0.04
[60 | 71.19] loss=0.01 avg=0.03
[70 | 83.99] loss=0.01 avg=0.03
[80 | 96.81] loss=0.01 avg=0.02
[90 | 109.66] loss=0.01 avg=0.02
[100 | 122.46] loss=0.01 avg=0.02
Saving checkpoint/finetune_0/model-100
[110 | 138.35] loss=0.01 avg=0.02
Saving checkpoint/finetune_0/model-110
Instructions for updating:
Use standard file APIs to delete files with this prefix.
/content/drive/My Drive/GRIMM/THE_STRAW_THE_COAL_AND_THE_BEAN.txt


ValueError: ignored

My first attempt is going to go like this:
- Run finetune on individual stories with 124m model and 100 steps